In [55]:
import pickle
import pandas as pd

In [56]:
with open('pickle/pdf_scrape.pickle', 'rb') as to_read: 
    pdf_scrape = pickle.load(to_read)

In [57]:
pdf_scrape.head()

,api_num,choke,comp_date,comp_type,ip24_gas,ip24_oil,ip24_water,ll,md,purpose,test_date,test_method,tvd,well_type
0,42-317-40950,Oil produced prior to test:,08/15/2017,Other/Recompletion,Casing Pressure:,Oil Gravity - API - 60.:,None,9926.0,19455,Well Record Only,None,Choke size:,9136,Producing
1,42-329-41745,128,06/14/2017,New Well,920,832.0,2104,NaN,19220,Initial Potential,07/01/2018,Flowing,9757,Producing
2,42-227-39057,Oil produced prior to test:,03/17/2017,New Well,Casing Pressure:,Oil Gravity - API - 60.:,None,10374.0,19038,Well Record Only,None,Choke size:,8373,Producing
3,42-227-39327,Was swab used during this test?,10/20/2017,New Well,609,1782.0,1034,8811.0,16907,Initial Potential,05/23/2018,Pumping,7584,Producing
4,42-173-37674,64,07/21/2017,New Well,961,1031.0,1635,NaN,19849,Initial Potential,05/13/2018,Gas Lift,8811,Producing


In [58]:
pdf_scrape['api_num'] = pdf_scrape['api_num'].apply(lambda x: int(''.join(x.split('-'))+'0000'))

In [59]:
pdf_scrape.groupby(['purpose']).api_num.count().reset_index().sort_values(['api_num'],ascending=False)

,purpose,api_num
0,Initial Potential,745
1,Well Record Only,226


In [60]:
#well records do not have ip rates so let's drop those
pdf_scrape_ip = pdf_scrape[pdf_scrape['purpose'] == 'Initial Potential']

In [61]:
print(len(pdf_scrape_ip))
print(len(pdf_scrape_ip['api_num'].unique()))

745
743


In [62]:
pdf_scrape_ip.groupby(['api_num']).choke.count().reset_index().sort_values(['choke'],ascending=False).head()

,api_num,choke
360,42317410000000,3
488,42329411740000,1
490,42329412480000,1
491,42329412490000,1
492,42329412510000,1


In [63]:
pdf_scrape_ip[pdf_scrape_ip['api_num'] == 42317410000000]

,api_num,choke,comp_date,comp_type,ip24_gas,ip24_oil,ip24_water,ll,md,purpose,test_date,test_method,tvd,well_type
117,42317410000000,Was swab used during this test?,01/28/2019,New Well,689,974.0,5184,10268.0,19530,Initial Potential,06/01/2018,Pumping,8576,Producing
118,42317410000000,Was swab used during this test?,01/28/2019,New Well,689,974.0,5184,10268.0,19530,Initial Potential,06/01/2018,Pumping,8576,Producing
643,42317410000000,Was swab used during this test?,01/28/2019,New Well,689,974.0,5184,10268.0,19530,Initial Potential,06/01/2018,Pumping,8576,Producing


In [64]:
pdf_scrape_ip = pdf_scrape_ip.drop_duplicates().reset_index(drop=True)

In [65]:
pdf_scrape_ip.groupby(['api_num']).choke.count().reset_index().sort_values(['choke'],ascending=False).head()

,api_num,choke
0,42173343240000,1
499,42329412690000,1
490,42329412480000,1
491,42329412490000,1
492,42329412510000,1


In [66]:
pdf_scrape_ip['ip24_gas'] = pdf_scrape_ip['ip24_gas'].apply(pd.to_numeric, errors='coerce')
pdf_scrape_ip['ip24_oil'] = pdf_scrape_ip['ip24_oil'].apply(pd.to_numeric, errors='coerce')
pdf_scrape_ip['ip24_water'] = pdf_scrape_ip['ip24_water'].apply(pd.to_numeric, errors='coerce')
pdf_scrape_ip['ll'] = pdf_scrape_ip['ll'].apply(pd.to_numeric, errors='coerce')
pdf_scrape_ip['md'] = pdf_scrape_ip['md'].apply(pd.to_numeric, errors='coerce')
pdf_scrape_ip['tvd'] = pdf_scrape_ip['tvd'].apply(pd.to_numeric, errors='coerce')

In [67]:
pdf_scrape_ip['ll'] = pdf_scrape_ip['ll'].fillna(pdf_scrape_ip['md'] - pdf_scrape_ip['tvd'])

In [68]:
pdf_scrape_ip.head()

,api_num,choke,comp_date,comp_type,ip24_gas,ip24_oil,ip24_water,ll,md,purpose,test_date,test_method,tvd,well_type
0,42329417450000,128,06/14/2017,New Well,920,832.0,2104,9463.0,19220.0,Initial Potential,07/01/2018,Flowing,9757,Producing
1,42227393270000,Was swab used during this test?,10/20/2017,New Well,609,1782.0,1034,8811.0,16907.0,Initial Potential,05/23/2018,Pumping,7584,Producing
2,42173376740000,64,07/21/2017,New Well,961,1031.0,1635,11038.0,19849.0,Initial Potential,05/13/2018,Gas Lift,8811,Producing
3,42329419230000,Was swab used during this test?,08/28/2017,New Well,1534,1373.0,1277,7265.0,17280.0,Initial Potential,05/10/2018,Gas Lift,9811,Producing
4,42317409940000,128,09/19/2017,New Well,1102,1695.0,2157,11053.0,21200.0,Initial Potential,04/20/2018,Pumping,9772,Producing


In [73]:
pdf_scrape_ip.groupby(['well_type']).api_num.count().reset_index().sort_values(['api_num'],ascending=False)

,well_type,api_num
0,Producing,743


In [76]:
pdf_clean = pdf_scrape_ip[['api_num','ip24_oil','ip24_gas','ip24_water','ll','tvd','comp_type','test_method']]

In [77]:
with open('pickle/pdf_clean.pickle', 'wb') as to_write:
    pickle.dump(pdf_clean, to_write)